# First look at the SkySim5000 mass function

In [ ]:
import numpy as np
import pyccl as ccl
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs

%matplotlib inline

In [ ]:
skysim_cat = GCRCatalogs.load_catalog('skysim5000_v1.1.1_small')

In [ ]:
cosmo_ss  = skysim_cat.cosmology

In [ ]:
cosmo_ss

## Read in catalog of halo masses and redshifts, extracted from SkySim5000
NB: to be replaced with code that performs the extraction on the "small" catalog

In [ ]:
f = '/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Galaxy_Cluster_Catalogs_details/cosmoDC2/SkySim5000_DM_halos_catalog.pkl'
#f = '/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Galaxy_Cluster_Catalogs_details/cosmoDC2/SkySim5000_DM_halos.pkl'

data_ref = pkl.load(open(f,'rb'))
data = data_ref.copy()

In [ ]:
#plt.scatter(data['ra'], data['dec'], marker='.')

## Define a redshift and mass range for the comparison of data and prediction, and filter the data accordingly
NB: SkySim5000 M200c masses are in units of Msun/h

In [ ]:
zmin=0.2
zmax=0.8
mmin = 1.e11 # Msun
mmax = 1.e15 # Msun
filt1 = data['redshift'] > zmin
filt2 = data['redshift'] < zmax
data['baseDC2/sod_halo_mass'] = data_ref['baseDC2/sod_halo_mass']/cosmo_ss.h # sod_halo_mass in Msun/h, needs conversion
filt3 = data['baseDC2/sod_halo_mass'] > mmin
filt4 = data['baseDC2/sod_halo_mass'] < mmax
filt = filt1 * filt2 * filt3 * filt4


In [ ]:
nbins = 30
hist = plt.hist(np.log10(data['baseDC2/sod_halo_mass'][filt]), bins=nbins)
plt.yscale('log')
print(f"Total number of halos = {len(data['baseDC2/sod_halo_mass'][filt])}")
hist

## Prediction using CCL and the Tinker08 and Bocquet16 mass functions

In [ ]:
# CCL Cosmology
cosmo = ccl.Cosmology(Omega_c=cosmo_ss.Om0-cosmo_ss.Ob0, Omega_b=cosmo_ss.Ob0,
                      h=cosmo_ss.h, sigma8=cosmo_ss.sigma8, n_s=cosmo_ss.n_s, Neff=3.04)

print(cosmo)


### Differential comoving volume

In [ ]:
def dV_over_dOmega_dz(z):
    a = 1./(1. + z)
    da = ccl.background.angular_diameter_distance(cosmo, a) 
    E = ccl.background.h_over_h0(cosmo, a)
    return ((1+z)**2)*(da**2)*ccl.physical_constants.CLIGHT_HMPC/cosmo['h']/E 

In [ ]:
### Comparison to the volume computed with Astropy
dV_over_dOmega_dz(0.3), cosmo_ss.differential_comoving_volume(0.3).value

### CCL mass functions

In [ ]:
hmd_200c = ccl.halos.MassDef(200, 'critical')
def tinker08(logm, z):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncTinker08(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

hmd_200c = ccl.halos.MassDef200c()

def bocquet16(logm, z):
    mass = 10**(logm)
    hmf_200c = ccl.halos.MassFuncBocquet16(cosmo, mass_def=hmd_200c)
    nm = hmf_200c.get_mass_function(cosmo, mass, 1./(1+z))
    return nm # dn/dlog10M

In [ ]:
def integrand_tinker08(logm,z):
    return tinker08(logm, z)*dV_over_dOmega_dz(z)

def integrand_bocquet16(logm,z):
    return bocquet16(logm, z)*dV_over_dOmega_dz(z)

### Solid angle of the `image` catalog = 440 deg2

In [ ]:
DeltaOmega = 440 * np.pi**2/180**2

In [ ]:
N_predicted_T08 = []

# Loop over mass bins defined by plt.hist and predict the corresponding number of haloes
# for Tinker 08
for i in np.arange(len(hist[0])):
    logmmin = np.log10(10**hist[1][i]) 
    logmmax = np.log10(10**hist[1][i+1])
    N_T08 = scipy.integrate.dblquad(integrand_tinker08, zmin, zmax, lambda x:logmmin, lambda x:logmmax, epsabs=1.e-4, epsrel=1.e-4)
    N_predicted_T08.append(N_T08[0]) 

In [ ]:
N_predicted_B16 = []

# Loop over mass bins defined by plt.hist and predict the corresponding number of haloes
# for Bocquet 16
for i in np.arange(len(hist[0])):
    logmmin = np.log10(10**hist[1][i]) # Msun
    logmmax = np.log10(10**hist[1][i+1]) # Msun
    N_B16 = scipy.integrate.dblquad(integrand_bocquet16, zmin, zmax, lambda x:logmmin, lambda x:logmmax, epsabs=1.e-5, epsrel=1.e-4)
    N_predicted_B16.append(N_B16[0]) 

## Plot measured versus predicted number of haloes

In [ ]:
plt.figure(figsize=(10,6))
hist = plt.hist(np.log10(data['baseDC2/sod_halo_mass'][filt]), bins=nbins, 
                label=f'SkySim5000, 440 deg2 field, z=[{zmin},{zmax}]',histtype='bar', alpha=0.5)

bin_centers=[(hist[1][i]+hist[1][i+1])/2. for i in np.arange(len(hist[0]))]
plt.plot(bin_centers,np.array(N_predicted_T08)*DeltaOmega, 'x-', label='T08, CCL')
plt.plot(bin_centers,np.array(N_predicted_B16)*DeltaOmega, '.-', label='B16, CCL')
plt.legend()
plt.yscale('log')
plt.xlabel('$M_{200c}$ [M$_\odot$]')
plt.ylabel('Number of haloes')